In [1]:
import gc
import sys
import string
from time import strftime
import json
from collections import defaultdict
import torch
from torch import nn
from torch import optim
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import ParameterGrid, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from collections import defaultdict
import os
import glob
import random

In [2]:
from IPython.utils.io import Tee

# Redirect all the outputs messages to the terminal and to a log file
logs_dir = './logs'
logfilename = logs_dir + strftime('/ipython_%Y-%m-%d_%H:%M:%S') + '.log' 
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)
    
sys.stdout = open('/dev/stdout', 'w')
Tee(logfilename, mode='w', channel='stdout')


## Configuration

In [2]:
random.seed(0)

JOINT = 'Knee'
FORCE_CELLS_PER_JOINT = {
    'Hip': [5, 6],
    'Knee': [3, 4, 7, 8],
    'Ankle': [1, 2]
}

CELLS = FORCE_CELLS_PER_JOINT[JOINT]

# Directory where the derived data is stored
DERIVED_DATA_DIR = '../../../../data'
# Path where the results are stored
RESULTS_PATH = '../../../../results'
# Total number of experiments
N_EXPERIMENTS = 63
# Number of folds for cross-validation
CV = 6
# Experiments for training set (int)
TRAIN_SIZE = 54
# Experiments for test set (int)
TEST_SIZE = 9

# ID of the training and validation data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0012_09082021'
# Hyperparameters search date
hs_date = '16082021'

# Parameters grid
param_grid = {
    'sequence_len': [5],
    'hidden_dim': [32],
    'sigma': [1],
    'n_layers': [1]
}

TORCH_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', TORCH_DEVICE)

Using device: cpu


/home/hl0d0w1g/data/Educación/Master Ing de la Informacion para la Salud/Master Thesis/EXOSAFE-ML/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


## Load data

In [3]:
experiments_dirs_path = glob.glob(DERIVED_DATA_DIR + '/*/*')
print('Number of experiments:', len(experiments_dirs_path))

Number of experiments: 63


In [ ]:
# Experiment selection (optional)
experiments_dirs_path_filter = []

exp_params = {}
for exp_path in experiments_dirs_path:
    with open(exp_path + '/parameters.json') as f:
        exp_params[exp_path] = json.load(f)
        
    if int(exp_params[exp_path]['SkinConfig']) == 0 or exp_params[exp_path]['SkinConfig'] == 'NaN':
        experiments_dirs_path_filter.append(exp_path)
        
print('Final number of experiments:', len(experiments_dirs_path_filter))
experiments_dirs_path = experiments_dirs_path_filter

# Redefine the split of the data for the filtered experiments

# Total number of experiments
N_EXPERIMENTS = 40
# Number of folds for cross-validation
CV = 4
# Experiments for training set (int)
TRAIN_SIZE = 32
# Experiments for test set (int)
TEST_SIZE = 8

assert(TRAIN_SIZE + TEST_SIZE == N_EXPERIMENTS)
assert(TRAIN_SIZE % CV == 0)

In [4]:
H3_LEG = 'L' # L|R

# features = [H3_LEG + a + m for a in ['Hip', 'Knee', 'Ankle'] for m in ['Pos', 'Vel', 'Acc', 'Torque']] + ['LegKnee{}Filtered'.format(m) for m in ['Position', 'Velocity', 'Torque']]
# targets = ['F' + str(i + 1) + ax for i in range(N_CELLS) for ax in ['x', 'y', 'z']]
features = ['L{}Pos'.format(JOINT)] + ['F' + str(i) + 'z' for i in FORCE_CELLS_PER_JOINT[JOINT]]
targets = ['F' + str(i) + ax for i in FORCE_CELLS_PER_JOINT[JOINT] for ax in ['x', 'y']]

print('Number of features: {}'.format(len(features)))
print('Selected features: {}'.format(features))
print('\n')
print('Number of targets: {}'.format(len(targets)))
print('Selected targets: {}'.format(targets))

Number of features: 5
Selected features: ['LKneePos', 'F3z', 'F4z', 'F7z', 'F8z']


Number of targets: 8
Selected targets: ['F3x', 'F3y', 'F4x', 'F4y', 'F7x', 'F7y', 'F8x', 'F8y']


In [5]:
targets_dict = {}
features_dict = {}
for i, exp_path in enumerate(experiments_dirs_path):
    print('{} - Experiment {} from {}'.format(i, exp_path.split('/')[-1], exp_path.split('/')[-2]))
    
    # Load targets
    targets_df = pd.read_csv(exp_path + '/force_cells_processed.csv')
    
    # Load features
    exo_df = pd.read_csv(exp_path + '/H3_processed.csv')
    # leg_df = pd.read_csv(exp_path + '/leg_processed.csv')
    # features_df = pd.concat([exo_df, leg_df], axis=1)
    features_df = exo_df
    
    idx_aux = targets_df.duplicated(keep='first')
    targets_df = targets_df.loc[~idx_aux]
    features_df = features_df.loc[~idx_aux]
    print('Droping {} duplicated data points'.format(len(idx_aux[idx_aux == False])))
    
    # Drop first row to remove noise in the start of the data recording
    targets_df = targets_df.iloc[1:]
    features_df = features_df.iloc[1:]
    # Drop null values
    idx = features_df.notna().all(axis=1)
    features_df = features_df.loc[idx]
    targets_df = targets_df.loc[idx]
    print('Droping {} data points by null features'.format(len(idx[idx == False])))

    assert(len(features_df) == len(targets_df))
    data_df = pd.concat([features_df, targets_df], axis=1)
    
    # Store the final array
    targets_dict[i] = data_df[targets].values
    features_dict[i] = data_df[features].values
    
    print('Experiment {} -> X: {}, Y: {} \n'.format(i, features_dict[i].shape, targets_dict[i].shape))

0 - Experiment 1 from 10032021
Droping 2917 duplicated data points
Droping 0 data points by null features
Experiment 0 -> X: (2916, 5), Y: (2916, 8) 

1 - Experiment 1 from 16022021
Droping 8709 duplicated data points
Droping 0 data points by null features
Experiment 1 -> X: (8708, 5), Y: (8708, 8) 

2 - Experiment 2 from 16022021
Droping 8696 duplicated data points
Droping 0 data points by null features
Experiment 2 -> X: (8695, 5), Y: (8695, 8) 

3 - Experiment 3 from 16022021
Droping 8708 duplicated data points
Droping 0 data points by null features
Experiment 3 -> X: (8707, 5), Y: (8707, 8) 

4 - Experiment 4 from 16022021
Droping 8736 duplicated data points
Droping 0 data points by null features
Experiment 4 -> X: (8735, 5), Y: (8735, 8) 

5 - Experiment 5 from 16022021
Droping 8706 duplicated data points
Droping 0 data points by null features
Experiment 5 -> X: (8705, 5), Y: (8705, 8) 

6 - Experiment 6 from 16022021
Droping 8680 duplicated data points
Droping 0 data points by nu

In [6]:
experiments = list(range(N_EXPERIMENTS))
random.shuffle(experiments)

train_experiments = experiments[:TRAIN_SIZE]
test_experiments = experiments[TRAIN_SIZE:]

print('Train experiments ids ({}): {}'.format(len(train_experiments), train_experiments))
print('Test experiments ids ({}): {}'.format(len(test_experiments), test_experiments))

assert(len(train_experiments) + len(test_experiments) == N_EXPERIMENTS)
# Check that no test experiment is in train
assert(not any([i in test_experiments for i in train_experiments]))

Train experiments ids (54): [53, 41, 3, 60, 33, 58, 27, 5, 7, 44, 49, 28, 23, 29, 46, 12, 57, 0, 61, 1, 43, 40, 14, 15, 17, 62, 20, 36, 10, 47, 11, 35, 52, 21, 4, 42, 51, 9, 38, 34, 59, 39, 6, 45, 18, 8, 55, 13, 37, 22, 30, 19, 50, 25]
Test experiments ids (9): [31, 32, 16, 2, 26, 56, 48, 24, 54]


## RNN hyperparameters search

In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_dim, sigma, output_size, sequence_len, num_iter, n_layers, device, lr=0.001, batch_size=32, early_stopping_patience=10):
        
        # input size -> Dimension of the input signal
        # outpusize -> Dimension of the output signal
        # hidden_dim -> Dimension of the rnn state
        # n_layers -> If >1, we are using a stacked RNN
        
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.input_size = input_size
        self.sigma = sigma
        self.output_size = output_size
        self.sequence_length = sequence_len
        self.num_layers = n_layers
        
        # define an RNN with specified parameters
        # batch_first=True means that the first dimension of the input will be the batch_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_dim, num_layers=n_layers, 
                          nonlinearity='relu', batch_first=True)
        
        # last, fully-connected layer
        self.fc1 = nn.Linear(hidden_dim, output_size) # YOUR CODE HERE

        
        self.lr = lr #Learning Rate
        self.batch_size = batch_size
        self.optim = optim.Adam(self.parameters(), self.lr)
        self.num_iter = num_iter
        self.early_stopping_patience = early_stopping_patience
        self.criterion = nn.MSELoss() #YOUR CODE HERE     
        
        self.device = device
        self.to(self.device)
        
        # A list to store the loss evolution along training
        self.loss_during_training = [] 
        self.valid_loss_during_training = []
        
    def forward(self, x, h0=None):
        
        '''
        About the shape of the different tensors ...:
        
        - Input signal x has shape (batch_size, seq_length, input_size)
        - The initialization of the RNN hidden state h0 has shape (n_layers, batch_size, hidden_dim).
          If None value is used, internally it is initialized to zeros.
        - The RNN output (batch_size, seq_length, hidden_size). This output is the RNN state along time  

        '''
        batch_size = x.size(0) # Number of signals N
        seq_length = x.size(1) # T
        
        # get RNN outputs
        # r_out is the sequence of states
        # hidden is just the last state (we will use it for forecasting)
        #print(x.shape)
        r_out, hidden = self.rnn(x, h0)
        #print(r_out.shape, hidden.shape)
        
        # shape r_out to be (seq_length, hidden_dim) #UNDERSTANDING THIS POINT IS IMPORTANT!!        
        r_out = r_out.reshape(-1, self.hidden_dim) 
        #print(r_out.shape)
        
        output = self.fc1(r_out)
        #print(output.shape)
        
        noise = torch.randn_like(output) * self.sigma
        
        output += noise
        
        # reshape back to temporal structure
        output = output.reshape([-1, self.sequence_length, 1])

        return output, hidden
           
    def trainloop(self, x, y, x_val=None, y_val=None):
        
        epochs_wo_improvement = self.early_stopping_patience
        # SGD Loop
        for e in range(int(self.num_iter)):
        
            running_loss = 0.
            
            idx = list(range(x.shape[0]))
            random.shuffle(idx)
            
            batchs = [idx[i:i + self.batch_size] for i in range(0, x.shape[0], self.batch_size)]
            for b in batchs:

                features = x[b, :, :]
                target = y[b, :, :]

                # print(features.shape, target.shape)
                self.optim.zero_grad() 
                
                features = torch.Tensor(features).to(self.device) #YOUR CODE HERE  
                target = torch.Tensor(target).to(self.device) #YOUR CODE HERE  
                
                out,hid = self.forward(features)

                # print(out.shape, hid.shape, target.shape)
                # print(out[:, -1, 0].size(), target[:, 0].size())
                loss = self.criterion(out[:, -1, 0], target[:, -1, 0]) #YOUR CODE HERE
                running_loss += loss.item()

                loss.backward()

                # This code helps to avoid vanishing exploiting gradients in RNNs
                # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
                nn.utils.clip_grad_norm_(self.parameters(), 2.0)

                self.optim.step()
                
            self.loss_during_training.append(running_loss / len(batchs))
            
            if x_val is not None and y_val is not None:
                with torch.no_grad():
                    # Set the model in evaluation mode 
                    self.eval()

                    running_loss_val = 0.
                    idx = list(range(x_val.shape[0]))

                    batchs = [idx[i:i + self.batch_size] for i in range(0, x_val.shape[0], self.batch_size)]
                    for b in batchs:
                        features = x_val[b, :, :]
                        target = y_val[b, :, :]

                        features = torch.Tensor(features).to(self.device) #YOUR CODE HERE  
                        target = torch.Tensor(target).to(self.device) #YOUR CODE HERE  

                        out,hid = self.forward(features)

                        # print(out.shape, hid.shape, target.shape)
                        # print(out[:, -1, 0].size(), target[:, -1, 0].size())
                        loss = self.criterion(out[:, -1, 0], target[:, -1, 0]) #YOUR CODE HERE
                        running_loss_val += loss.item()

                    self.valid_loss_during_training.append(running_loss_val / len(splits))

                # Return the model to training mode
                self.train()

                print("%d epochs -> Train loss: %f - Val loss: %f"%(e, self.loss_during_training[-1], self.valid_loss_during_training[-1]))
                
                if len(self.valid_loss_during_training) > self.early_stopping_patience + 1:
                    if self.valid_loss_during_training[-1] > np.mean(self.valid_loss_during_training[-self.early_stopping_patience:]):
                        print('Early stopping at epoch', e + 1)
                        return e
            else:
                print("%d epochs -> Train loss: %f"%(e, self.loss_during_training[-1]))

In [8]:
param_grid_ls = list(ParameterGrid(param_grid))
random.shuffle(param_grid_ls)
param_grid_len = len(param_grid_ls)
print('Number of parameters combinations: {}'.format(param_grid_len))

for idx, params in enumerate(param_grid_ls):
    params_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
    print('Parameters ({}) {}/{}  -  {}'.format(params_id, idx + 1, param_grid_len, strftime('%Y-%m-%d %H:%M:%S')))
    print(params)
    
    # Prepare sequences
    X_train, Y_train = [], [] 
    for i in train_experiments:
        X_aux = features_dict[i]
        Y_aux = targets_dict[i]

        splits = [i for i in range(0, X_aux.shape[0], params['sequence_len'])]

        X_tensor = np.zeros([len(splits), params['sequence_len'], len(features)])
        Y_tensor = np.zeros([len(splits), params['sequence_len'], len(targets)])

        for idx, i in enumerate(splits[:-1]):
            X_tensor[idx, :, :] = X_aux[splits[idx]:splits[idx + 1], :]
            Y_tensor[idx, :, :] = Y_aux[splits[idx + 1] - 1, :]

        # print(X_tensor.shape)
        # print(Y_tensor.shape)

        X_train.append(X_tensor)
        Y_train.append(Y_tensor)

    X_train, Y_train = np.concatenate(X_train, axis=0), np.concatenate(Y_train, axis=0) 


    X_test, Y_test = [], [] 
    for i in test_experiments:
        X_aux = features_dict[i]
        Y_aux = targets_dict[i]

        splits = [i for i in range(0, X_aux.shape[0], params['sequence_len'])]

        X_tensor = np.zeros([len(splits), params['sequence_len'], len(features)])
        Y_tensor = np.zeros([len(splits), params['sequence_len'], len(targets)])

        for idx, i in enumerate(splits[:-1]):
            X_tensor[idx, :, :] = X_aux[splits[idx]:splits[idx + 1], :]
            Y_tensor[idx, :, :] = Y_aux[splits[idx + 1] - 1, :]

        # print(X_tensor.shape)
        # print(Y_tensor.shape)

        X_test.append(X_tensor)
        Y_test.append(Y_tensor)

    X_test, Y_test = np.concatenate(X_test, axis=0), np.concatenate(Y_test, axis=0) 

    X_train_norm = np.zeros(list(X_train.shape))
    X_test_norm = np.zeros(list(X_test.shape))

    scalers = []
    for f in range(len(features)):
        s = MinMaxScaler().fit(X_train[:, :, f])

        scalers.append(s)

        X_train_norm[:, :, f] = s.transform(X_train[:, :, f])
        X_test_norm[:, :, f] = s.transform(X_test[:, :, f])


    results = defaultdict(list)
    tr_time = []
    early_stopping = []
    for t in range(Y_train.shape[2]):

        model = RNN(input_size=X_train_norm.shape[2], output_size=1, num_iter=50, device=TORCH_DEVICE, **params)

        t_start = time.time()
        last_epoch = model.trainloop(X_train_norm, Y_train[:, :, t:t+1], X_test_norm, Y_test[:, :, t:t+1])
        tr_time.append(time.time() - t_start)
        early_stopping.append(last_epoch)

        all_results = {'MAE': [], 'MSE': [], 'R2': []}
        for i in test_experiments:
            X_aux = features_dict[i]
            Y_aux = targets_dict[i]

            true = []
            pred = []

            for i in range(params['sequence_len'], X_aux.shape[0]):
                X_aux_ = X_aux[np.newaxis, i-params['sequence_len']:i, :]
                for f in range(len(features)):
                    s = scalers[f]

                    X_aux_[:, :, f] = s.transform(X_aux_[:, :, f])

                features_ = torch.Tensor(X_aux_).to(TORCH_DEVICE)
                target_ = torch.Tensor(Y_aux[np.newaxis, i, t:t+1]).to(TORCH_DEVICE)

                out, hid = model.forward(features_)

                true.append(target_[:, 0].cpu().detach().numpy())
                pred.append(out[:, -1, 0].cpu().detach().numpy())

            all_results['MAE'].append(mean_absolute_error(true, pred))
            all_results['MSE'].append(mean_squared_error(true, pred))
            all_results['R2'].append(r2_score(true, pred))

        # results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
        # results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
        # results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
        results['Valid_MAE'].append(np.mean(all_results['MAE']))
        results['Valid_MSE'].append(np.mean(all_results['MSE']))
        results['Valid_R2'].append(np.mean(all_results['R2']))

    results['early_stopping'] = early_stopping
    results['fit_time'] = sum(tr_time)
    print('Training time: {:.4f}'.format(results['fit_time']))

    for subset in ['Valid']:
        for f, force in enumerate(['Fx', 'Fy']):
            for loss in ['MAE', 'MSE', 'R2']:
                scores = [results['_'.join([subset, loss])][i + f] for i in range(0, len(CELLS) * 2, 2)]
                results['_'.join([subset, force, loss, 'mean'])].append(np.mean(scores))
                results['_'.join([subset, force, loss, 'std'])].append(np.std(scores))

    # Save the obtained results and its parameters into a JSON file
    rd = {}
    rd['id'] = params_id
    rd['parameters'] = params
    rd['results'] = dict(results)
    
    save_dir = os.path.join(RESULTS_PATH, DATA_ID, '{}_RNN_{}'.format(JOINT, hs_date))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    with open(os.path.join(save_dir, '{}_RNN_{}_{}.json'.format(JOINT, hs_date, params_id)), 'w') as fp:
        json.dump(rd, fp)
    
    print('\n\n')
    del model, results, rd
    gc.collect()

Number of parameters combinations: 1
Parameters (I82QCLS7DT) 1/1  -  2021-08-15 20:05:57
{'hidden_dim': 32, 'n_layers': 1, 'sequence_len': 5, 'sigma': 1}
0 epochs -> Train loss: 375.860282 - Val loss: 146.279718
1 epochs -> Train loss: 271.087544 - Val loss: 134.778701
2 epochs -> Train loss: 264.720525 - Val loss: 138.338348
3 epochs -> Train loss: 246.643673 - Val loss: 105.755041
4 epochs -> Train loss: 212.816215 - Val loss: 103.600939
5 epochs -> Train loss: 201.100178 - Val loss: 104.950213
6 epochs -> Train loss: 191.990312 - Val loss: 93.546279
7 epochs -> Train loss: 183.940083 - Val loss: 82.502934
8 epochs -> Train loss: 178.096340 - Val loss: 93.873808
9 epochs -> Train loss: 171.284014 - Val loss: 93.499993
10 epochs -> Train loss: 164.119390 - Val loss: 82.754420
11 epochs -> Train loss: 156.595714 - Val loss: 73.375633
12 epochs -> Train loss: 152.523878 - Val loss: 68.053040
13 epochs -> Train loss: 150.583894 - Val loss: 100.296576
Early stopping at epoch 14
0 epochs -

TypeError: Object of type float32 is not JSON serializable